In [0]:
%pip install python-dotenv

In [0]:
from dotenv import load_dotenv
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, DateType
from pyspark.sql.window import Window
import requests
import time
from bs4 import BeautifulSoup
from datetime import datetime
from typing import Optional

In [0]:
catalog_dev = "`land_topografisk-gdb_dev`"
schema_dev = "ai2025"
spark.sql(f"USE CATALOG {catalog_dev}")
spark.sql(f"USE SCHEMA {schema_dev}")
bronze_table = "endepunkt_bronze"
silver_table = "endepunkt_silver"
buffer = 64

BASE_PATH = "/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER"

In [0]:
def generate_dom_url(bbox):
    """
    Genererer en URL for en DOM-bilde basert på bbox.
    """
    bbox_str = ",".join(map(str, bbox))
    width, height = 512, 512
    resolution = 0.2
    return (
        f"https://wms.geonorge.no/skwms1/wms.hoyde-dom-nhm-25833?request=GetMap&Format=image/png&"
        f"GetFeatureInfo=text/plain&CRS=EPSG:25833&Layers=NHM_DOM_25833:skyggerelieff&"
        f"BBOX={bbox_str}&width={width}&height={height}&RESOLUTION={resolution}"
    )


generate_dom_url_udf = udf(generate_dom_url, StringType())

In [0]:
def dom_file_exists(nodeid: str) -> str:
    """
    Sjekker om DOM-filen eksisterer for en gitt nodeid.
    """
    path = f"/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/storreendepunkt_dom/dom_{nodeid}.png"
    return "DOWNLOADED" if os.path.exists(path) else "PENDING"


dom_file_status_udf = udf(dom_file_exists, StringType())

In [0]:
def generate_image_url(bbox):
    """
    Genererer en URL for et ortofoto-bilde basert på bbox.
    """
    try:
        bbox_str = ",".join(map(str, bbox))
        width, height = 512, 512
        resolution = 0.2
        return (
            f"https://wms.geonorge.no/skwms1/wms.nib?VERSION=1.3.0"
            f"&service=WMS&request=GetMap&Format=image/png&"
            f"GetFeatureInfo=text/plain&CRS=EPSG:25833&Layers=ortofoto&"
            f"BBox={bbox_str}&width={width}&height={height}&RESOLUTION={resolution}"
        )
    except Exception as e:
        return "INVALID"


generate_image_url_udf = udf(generate_image_url, StringType())

In [0]:
def image_file_exists(nodeid: str) -> str:
    """
    Sjekker om ortofoto-filen eksisterer for en gitt nodeid.
    """
    path = f"/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/storreendepunkt_images/image_{nodeid}.png"
    return "DOWNLOADED" if os.path.exists(path) else "PENDING"


image_file_status_udf = udf(image_file_exists, StringType())

In [0]:
load_dotenv()
BRUKERID = os.getenv("GEONORGE_BRUKERID")
PASSORD = os.getenv("GEONORGE_PASSORD")


def get_token():
    """
    Hender token fra GeoNorge og returnerer det.
    """
    url = (
        f"https://baat.geonorge.no/skbaatts/req?brukerid={BRUKERID}"
        f"&passord={PASSORD}&tjenesteid=wms.nib&retformat=s"
    )
    raw_token = requests.get(url).text.strip("`")
    return raw_token


token = get_token()
token_start_time = time.time()
token_lifetime = 55 * 60


def refresh_token_if_needed():
    """
    henter ny token om den gamle er utløpt.
    """
    global token, token_start_time
    if time.time() - token_start_time > token_lifetime:
        print("🔄 Fornyer token...")
        token = get_token()
        token_start_time = time.time()

In [0]:
def get_fotodato(bbox: str, token: str, max_retries=10):
    """
    Henter fotodato for en bbox.
    """
    url = f"https://wms.geonorge.no/skwms1/wms.nib?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetFeatureInfo&CRS=EPSG:25833&BBOX={bbox}&WIDTH=512&HEIGHT=512&LAYERS=ortofoto&QUERY_LAYERS=ortofoto&INFO_FORMAT=text/html&I=256&J=256&TICKET={token}"

    table = None
    field_value = None
    for i in range(max_retries):
        try:
            response = requests.get(url, timeout=10)
            time.sleep(1.0)
            soup = BeautifulSoup(response.text, "html.parser")

            table = soup.find("table")
            break
        except Exception as e:
            wait = 2**i
            print(f"⚠️ Feil ved henting av fotodato ({e}), prøver igjen om {wait}s...")
            time.sleep(wait)

    if table:
        rows = table.find_all("tr")
        for row in rows:
            cells = row.find_all("td")
            if len(cells) >= 2 and cells[0].text.strip() == "Fotodato":
                field_value = cells[1].text.strip()
                field_value = datetime.strptime(field_value, "%d.%m.%Y").date()
                return field_value

    return None

In [0]:
def add_ortofoto_date(df, token: str):
    """
    Legger til fotodato kolonnen til en DataFrame.
    """
    # Henter bare relevante kolonner
    sample_rows = df.select("nodeid", "bbox_str").collect()

    # Henter fotodato
    bbox_date_pairs = [
        (row["nodeid"], get_fotodato(row["bbox_str"].replace("_", ","), token))
        for row in sample_rows
    ]

    schema = StructType(
        [
            StructField("nodeid", StringType(), True),
            StructField("fototid", DateType(), True),
        ]
    )

    bbox_date_df = spark.createDataFrame(bbox_date_pairs, schema)
    df_with_date = df.join(bbox_date_df, on="nodeid", how="left")

    return df_with_date

In [0]:
def add_silver_columns(
    df: DataFrame, buffer: Optional[int], kommune_id: str = ""
) -> DataFrame:
    """
    Legger til kolonner til en DataFrame med buffer, bbox, bbox_str, image_wms, og dom_wms.
    """
    buffer = buffer or 64
    df = df.withColumn(
        "bbox", expr(f"array(x - {buffer}, y - {buffer}, x + {buffer}, y + {buffer})")
    )
    df = df.withColumn("bbox_str", concat_ws(",", col("bbox")))
    df = (
        df.withColumn("image_wms", generate_image_url_udf(col("bbox")))
        .withColumn("dom_wms", generate_dom_url_udf(col("bbox")))
        .withColumn("image_status", image_file_status_udf(col("nodeid")))
        .withColumn("dom_status", dom_file_status_udf(col("nodeid")))
        .withColumn("lastet_tid", current_timestamp())
        .withColumn("kommune_id", lit(kommune_id))
    )
    df = df.withColumn(
        "image_path",
        concat(
            lit(BASE_PATH),
            lit("/storreendepunkt_images/image_"),
            col("nodeid"),
            lit(".png"),
        ),
    )
    df = df.withColumn(
        "dom_path",
        concat(
            lit(BASE_PATH), lit("/storreendepunkt_dom/dom_"), col("nodeid"), lit(".png")
        ),
    )
    df = add_ortofoto_date(df, token)
    return df

In [0]:
def write_delta_table(sdf: DataFrame, mode: str = "merge") -> None:
    """
    Skriver data til deltatabellen og opdaterer dersom den nodeid allerede finnes.
    """
    if mode == "overwrite":
        sdf.write.format("delta").option("mergeSchema", "true").mode(
            "overwrite"
        ).saveAsTable(silver_table)
    else:
        from delta.tables import DeltaTable

        delta_tbl = DeltaTable.forName(spark, silver_table)

        delta_tbl.alias("target").merge(
            sdf.alias("source"), condition="target.nodeid = source.nodeid"
        ).whenMatchedUpdate(
            condition="target.hentet_tid < source.hentet_tid OR target.image_path IS NULL",
            set={col: f"source.{col}" for col in sdf.columns},
        ).whenNotMatchedInsert(
            values={col: f"source.{col}" for col in sdf.columns}
        ).execute()

In [0]:
def process_silver_for_kommune(kommune_id: str) -> None:
    """
    Prosesserer data for en gitt kommune og skriver til deltatabellen.
    """
    bronze_df = spark.read.table(bronze_table)
    bronze_df = bronze_df.filter(col("kommune_id") == lit(kommune_id))

    silver_df = add_silver_columns(bronze_df, buffer=buffer, kommune_id=kommune_id)

    if not spark.catalog.tableExists(silver_table):
        write_delta_table(silver_df, mode="overwrite")
        return

    expected_schema = spark.table(silver_table).schema

    silver_df = silver_df.select(
        [
            lit("").cast("string").alias(c.name)
            if c.dataType.typeName() == "void"
            else col(c.name).cast(c.dataType)
            for c in expected_schema
        ]
    )

    w = Window.partitionBy("nodeid").orderBy(col("hentet_tid").desc())
    silver_df = (
        silver_df.withColumn("row_number", row_number().over(w))
        .filter(col("row_number") == lit(1))
        .drop("row_number")
    )

    write_delta_table(silver_df)

In [0]:
kommune_id_rows = [
    row.asDict()
    for row in spark.read.table(bronze_table).select("kommune_id").distinct().collect()
]  # Henter ut alle kommuner fra bronsetabellen

for row in kommune_id_rows:
    kommune_id = str(row["kommune_id"])
    print(f"Prosesserer kommune {kommune_id}...")
    if kommune_id is not None:
        process_silver_for_kommune(kommune_id)